# Forecasting Electricity Prices in Poland (RCE)

## Import Libraries

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


## Loading Data

In [15]:
# Step 0: imports (keep it simple)
import pandas as pd

# Step 1: use RAW urls (NOT github blob pages)
train_urls = [
    "https://raw.githubusercontent.com/anlan-chen/ML2-Final-Project/main/Data/Data/Training/GUI_ENERGY_PRICES_201912312300-202012312300.csv",
    "https://raw.githubusercontent.com/anlan-chen/ML2-Final-Project/main/Data/Data/Training/GUI_ENERGY_PRICES_202012312300-202112312300.csv",
    "https://raw.githubusercontent.com/anlan-chen/ML2-Final-Project/main/Data/Data/Training/GUI_ENERGY_PRICES_202112312300-202212312300.csv",
    "https://raw.githubusercontent.com/anlan-chen/ML2-Final-Project/main/Data/Data/Training/GUI_ENERGY_PRICES_202312312300-202412312300.csv",
]

test_urls = [
    "https://raw.githubusercontent.com/anlan-chen/ML2-Final-Project/main/Data/Data/Testing/GUI_ENERGY_PRICES_202412312300-202512312300.csv",
]

# Step 2: load training data
train_list = []
for url in train_urls:
    df = pd.read_csv(url)
    df["source_url"] = url
    train_list.append(df)

train_df = pd.concat(train_list, ignore_index=True)

print("train_df shape:", train_df.shape)
print("train_df columns:", train_df.columns.tolist())
display(train_df.head())

# Step 3: load testing data
test_list = []
for url in test_urls:
    df = pd.read_csv(url)
    df["source_url"] = url
    test_list.append(df)

test_df = pd.concat(test_list, ignore_index=True)

print("test_df shape:", test_df.shape)
print("test_df columns:", test_df.columns.tolist())
display(test_df.head())


train_df shape: (35088, 7)
train_df columns: ['MTU (CET/CEST)', 'Area', 'Sequence', 'Day-ahead Price (EUR/MWh)', 'Intraday Period (CET/CEST)', 'Intraday Price (EUR/MWh)', 'source_url']


,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),Intraday Period (CET/CEST),Intraday Price (EUR/MWh),source_url
0,01/01/2020 00:00:00 - 01/01/2020 01:00:00,BZN|PL,Without Sequence,34.95,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
1,01/01/2020 01:00:00 - 01/01/2020 02:00:00,BZN|PL,Without Sequence,34.67,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
2,01/01/2020 02:00:00 - 01/01/2020 03:00:00,BZN|PL,Without Sequence,33.30,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
3,01/01/2020 03:00:00 - 01/01/2020 04:00:00,BZN|PL,Without Sequence,31.91,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
4,01/01/2020 04:00:00 - 01/01/2020 05:00:00,BZN|PL,Without Sequence,27.54,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...


test_df shape: (35040, 7)
test_df columns: ['MTU (CET/CEST)', 'Area', 'Sequence', 'Day-ahead Price (EUR/MWh)', 'Intraday Period (CET/CEST)', 'Intraday Price (EUR/MWh)', 'source_url']


,MTU (CET/CEST),Area,Sequence,Day-ahead Price (EUR/MWh),Intraday Period (CET/CEST),Intraday Price (EUR/MWh),source_url
0,01/01/2025 00:00:00 - 01/01/2025 00:15:00,BZN|PL,Without Sequence,15.41,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
1,01/01/2025 00:15:00 - 01/01/2025 00:30:00,BZN|PL,Without Sequence,15.41,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
2,01/01/2025 00:30:00 - 01/01/2025 00:45:00,BZN|PL,Without Sequence,15.41,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
3,01/01/2025 00:45:00 - 01/01/2025 01:00:00,BZN|PL,Without Sequence,15.41,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...
4,01/01/2025 01:00:00 - 01/01/2025 01:15:00,BZN|PL,Without Sequence,2.19,NaN,NaN,https://raw.githubusercontent.com/anlan-chen/M...


In [16]:
# Step 2: parse MTU into start_time and end_time
import pandas as pd

def parse_mtu_to_times(s):
    # Example: "01/01/2020 00:00:00 - 01/01/2020 01:00:00"
    left, right = s.split(" - ")
    start = pd.to_datetime(left, dayfirst=True, errors="coerce")
    end = pd.to_datetime(right, dayfirst=True, errors="coerce")
    return start, end

# training
train_times = train_df["MTU (CET/CEST)"].astype(str).apply(parse_mtu_to_times)
train_df["start_time"] = train_times.apply(lambda x: x[0])
train_df["end_time"]   = train_times.apply(lambda x: x[1])

# testing
test_times = test_df["MTU (CET/CEST)"].astype(str).apply(parse_mtu_to_times)
test_df["start_time"] = test_times.apply(lambda x: x[0])
test_df["end_time"]   = test_times.apply(lambda x: x[1])

# Step 3: keep only needed columns for now
y_col = "Day-ahead Price (EUR/MWh)"

train_clean = train_df[["start_time", "end_time", "Area", "Sequence", y_col]].copy()
test_clean  = test_df[["start_time", "end_time", "Area", "Sequence", y_col]].copy()

# Step 4: basic cleaning
train_clean = train_clean.dropna(subset=["start_time", y_col]).sort_values("start_time")
test_clean  = test_clean.dropna(subset=["start_time", y_col]).sort_values("start_time")

train_clean[y_col] = pd.to_numeric(train_clean[y_col], errors="coerce")
test_clean[y_col]  = pd.to_numeric(test_clean[y_col], errors="coerce")

print("train_clean shape:", train_clean.shape)
print("test_clean shape:", test_clean.shape)

print("train time range:", train_clean["start_time"].min(), "->", train_clean["start_time"].max())
print("test  time range:", test_clean["start_time"].min(), "->", test_clean["start_time"].max())


train_clean shape: (35072, 5)
test_clean shape: (35030, 5)
train time range: 2020-01-01 00:00:00 -> 2024-12-31 23:00:00
test  time range: 2025-01-01 00:00:00 -> 2025-12-31 23:45:00


In [17]:
# Step 5: check time step frequency
train_diff = train_clean["start_time"].diff().value_counts().head(5)
test_diff  = test_clean["start_time"].diff().value_counts().head(5)

print("Most common train time diffs:")
print(train_diff)

print("\nMost common test time diffs:")
print(test_diff)


Most common train time diffs:
start_time
0 days 01:00:00      35062
0 days 04:00:00          4
0 days 02:00:00          4
365 days 01:00:00        1
Name: count, dtype: int64

Most common test time diffs:
start_time
0 days 00:15:00    35027
0 days 01:45:00        1
0 days 01:15:00        1
Name: count, dtype: int64


In [18]:
# Step 6: set index and resample to 15 minutes
# For training: hourly -> 15min by forward fill
train_15m = (
    train_clean.set_index("start_time")
              .sort_index()
              .resample("15min")
              .ffill()
              .reset_index()
)

# For testing: already 15min, but we enforce consistent grid
test_15m = (
    test_clean.set_index("start_time")
             .sort_index()
             .resample("15min")
             .asfreq()   # keep original; do not fill target here
             .reset_index()
)

print("train_15m shape:", train_15m.shape)
print("test_15m shape:", test_15m.shape)

display(train_15m.head())
display(test_15m.head())


train_15m shape: (175389, 5)
test_15m shape: (35040, 5)


,start_time,end_time,Area,Sequence,Day-ahead Price (EUR/MWh)
0,2020-01-01 00:00:00,2020-01-01 01:00:00,BZN|PL,Without Sequence,34.95
1,2020-01-01 00:15:00,2020-01-01 01:00:00,BZN|PL,Without Sequence,34.95
2,2020-01-01 00:30:00,2020-01-01 01:00:00,BZN|PL,Without Sequence,34.95
3,2020-01-01 00:45:00,2020-01-01 01:00:00,BZN|PL,Without Sequence,34.95
4,2020-01-01 01:00:00,2020-01-01 02:00:00,BZN|PL,Without Sequence,34.67


,start_time,end_time,Area,Sequence,Day-ahead Price (EUR/MWh)
0,2025-01-01 00:00:00,2025-01-01 00:15:00,BZN|PL,Without Sequence,15.41
1,2025-01-01 00:15:00,2025-01-01 00:30:00,BZN|PL,Without Sequence,15.41
2,2025-01-01 00:30:00,2025-01-01 00:45:00,BZN|PL,Without Sequence,15.41
3,2025-01-01 00:45:00,2025-01-01 01:00:00,BZN|PL,Without Sequence,15.41
4,2025-01-01 01:00:00,2025-01-01 01:15:00,BZN|PL,Without Sequence,2.19


In [19]:
# Step 5.1: define target and copy data
target_col = "Day-ahead Price (EUR/MWh)"

train_fe = train_15m.copy()
test_fe  = test_15m.copy()


In [20]:
# Step 5.2: lag features (15-min resolution)
lag_list = [1, 4, 96, 672]

for lag in lag_list:
    train_fe[f"lag_{lag}"] = train_fe[target_col].shift(lag)
    test_fe[f"lag_{lag}"]  = test_fe[target_col].shift(lag)


In [21]:
# Step 5.3: rolling statistics
rolling_windows = [96, 672]

for w in rolling_windows:
    train_fe[f"roll_mean_{w}"] = train_fe[target_col].shift(1).rolling(w).mean()
    train_fe[f"roll_std_{w}"]  = train_fe[target_col].shift(1).rolling(w).std()

    test_fe[f"roll_mean_{w}"] = test_fe[target_col].shift(1).rolling(w).mean()
    test_fe[f"roll_std_{w}"]  = test_fe[target_col].shift(1).rolling(w).std()


In [22]:
# Step 5.4: calendar features
for df in [train_fe, test_fe]:
    df["hour"] = df["start_time"].dt.hour
    df["minute"] = df["start_time"].dt.minute
    df["dayofweek"] = df["start_time"].dt.dayofweek
    df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)


In [23]:
# Step 5.5: drop missing values caused by lags/rolling
train_fe = train_fe.dropna().reset_index(drop=True)
test_fe  = test_fe.dropna().reset_index(drop=True)

print("train_fe shape:", train_fe.shape)
print("test_fe shape:", test_fe.shape)


train_fe shape: (174621, 17)
test_fe shape: (33012, 17)


## Step 6：Prepare X / y and Baseline Model

In [24]:
# Step 6.1: define feature columns
feature_cols = [
    "lag_1",
    "lag_4",
    "lag_96",
    "lag_672",
    "roll_mean_96",
    "roll_std_96",
    "roll_mean_672",
    "roll_std_672",
    "hour",
    "minute",
    "dayofweek",
    "is_weekend"
]

target_col = "Day-ahead Price (EUR/MWh)"

print("Number of features:", len(feature_cols))
print(feature_cols)


Number of features: 12
['lag_1', 'lag_4', 'lag_96', 'lag_672', 'roll_mean_96', 'roll_std_96', 'roll_mean_672', 'roll_std_672', 'hour', 'minute', 'dayofweek', 'is_weekend']


In [25]:
# Step 6.2: split into X and y
X_train = train_fe[feature_cols]
y_train = train_fe[target_col]

X_test = test_fe[feature_cols]
y_test = test_fe[target_col]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (174621, 12)
X_test shape: (33012, 12)


In [26]:
# Step 6.3: naive baseline using lag_4 (1 hour)
y_pred_naive = X_test["lag_4"]

from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_naive = mean_absolute_error(y_test, y_pred_naive)
rmse_naive = mean_squared_error(y_test, y_pred_naive, squared=False)

print("Naive baseline results:")
print("MAE:", mae_naive)
print("RMSE:", rmse_naive)


Naive baseline results:
MAE: 16.298627468799225
RMSE: 27.52143615423511


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Step 7：LightGBM

In [27]:
# Step 7.1: prepare LightGBM datasets
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_test  = lgb.Dataset(X_test, y_test, reference=lgb_train)


In [28]:
# Step 7.2: LightGBM parameters (initial)
lgb_params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": ["l1", "l2"],
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1
}


In [29]:
# Step 7.3: train LightGBM
gbm = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=2000,
    valid_sets=[lgb_train],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)


Training until validation scores don't improve for 50 rounds


In [30]:
# Step 7.4: predict and evaluate
y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)

mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
rmse_lgb = mean_squared_error(y_test, y_pred_lgb, squared=False)

print("LightGBM results:")
print("MAE:", mae_lgb)
print("RMSE:", rmse_lgb)


LightGBM results:
MAE: 5.587601668186966
RMSE: 12.822283770553215


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Step 8：CatBoost (compare)

In [31]:
# Step 8.1: train CatBoost
from catboost import CatBoostRegressor

cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function="RMSE",
    verbose=False
)

cat_model.fit(X_train, y_train)


In [32]:
# Step 8.2: predict and evaluate
y_pred_cat = cat_model.predict(X_test)

mae_cat = mean_absolute_error(y_test, y_pred_cat)
rmse_cat = mean_squared_error(y_test, y_pred_cat, squared=False)

print("CatBoost results:")
print("MAE:", mae_cat)
print("RMSE:", rmse_cat)


CatBoost results:
MAE: 5.4589376937031195
RMSE: 12.617467671823402


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Step 9：Hyperparameter Tuning

In [33]:
# Step 9.1: create validation split from training data (last 20%)
split_idx = int(len(X_train) * 0.8)

X_tr = X_train.iloc[:split_idx]
y_tr = y_train.iloc[:split_idx]

X_val = X_train.iloc[split_idx:]
y_val = y_train.iloc[split_idx:]

print("Train part:", X_tr.shape)
print("Validation part:", X_val.shape)


Train part: (139696, 12)
Validation part: (34925, 12)


In [34]:
# Step 9.2: LightGBM tuning
lgb_train_tune = lgb.Dataset(X_tr, y_tr)
lgb_valid_tune = lgb.Dataset(X_val, y_val, reference=lgb_train_tune)

tuned_params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": ["l1", "l2"],
    "learning_rate": 0.03,
    "num_leaves": 63,
    "min_child_samples": 20,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1
}

gbm_tuned = lgb.train(
    tuned_params,
    lgb_train_tune,
    num_boost_round=4000,
    valid_sets=[lgb_valid_tune],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[492]	valid_0's l1: 4.34599	valid_0's l2: 117.605


In [35]:
# Step 9.2: LightGBM tuning
lgb_train_tune = lgb.Dataset(X_tr, y_tr)
lgb_valid_tune = lgb.Dataset(X_val, y_val, reference=lgb_train_tune)

tuned_params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": ["l1", "l2"],
    "learning_rate": 0.03,
    "num_leaves": 63,
    "min_child_samples": 20,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1
}

gbm_tuned = lgb.train(
    tuned_params,
    lgb_train_tune,
    num_boost_round=4000,
    valid_sets=[lgb_valid_tune],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[492]	valid_0's l1: 4.34599	valid_0's l2: 117.605


In [37]:
# Step 9.3: predict with tuned LightGBM
y_pred_lgb_tuned = gbm_tuned.predict(
    X_test, num_iteration=gbm_tuned.best_iteration
)

mae_lgb_tuned = mean_absolute_error(y_test, y_pred_lgb_tuned)
rmse_lgb_tuned = mean_squared_error(y_test, y_pred_lgb_tuned, squared=False)

print("Tuned LightGBM results:")
print("MAE:", mae_lgb_tuned)
print("RMSE:", rmse_lgb_tuned)


Tuned LightGBM results:
MAE: 7.026449971484944
RMSE: 14.980897284916976


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Step 10: 2025 Results Analysis (Peak / Trough / Underestimation)

In [38]:
# Step 10.1: build result dataframe
results_2025 = test_fe.copy()
results_2025["y_true"] = y_test.values
results_2025["y_pred_naive"] = y_pred_naive.values
results_2025["y_pred_lgb"] = y_pred_lgb
results_2025["y_pred_lgb_tuned"] = y_pred_lgb_tuned
results_2025["y_pred_cat"] = y_pred_cat

results_2025["error_lgb"] = results_2025["y_pred_lgb_tuned"] - results_2025["y_true"]

display(results_2025.head())


,start_time,end_time,Area,Sequence,Day-ahead Price (EUR/MWh),lag_1,lag_4,lag_96,lag_672,roll_mean_96,roll_std_96,roll_mean_672,roll_std_672,hour,minute,dayofweek,is_weekend,y_true,y_pred_naive,y_pred_lgb,y_pred_lgb_tuned,y_pred_cat,error_lgb
0,2025-01-08 00:00:00,2025-01-08 00:15:00,BZN|PL,Without Sequence,71.82,50.24,50.24,24.27,15.41,83.435417,37.176983,74.949286,46.804171,0,0,2,0,71.82,50.24,43.839942,48.761104,44.452715,-23.058896
1,2025-01-08 00:15:00,2025-01-08 00:30:00,BZN|PL,Without Sequence,71.82,71.82,50.24,24.27,15.41,83.930729,36.694038,75.033229,46.747780,0,15,2,0,71.82,50.24,70.070663,68.440114,71.070764,-3.379886
2,2025-01-08 00:30:00,2025-01-08 00:45:00,BZN|PL,Without Sequence,71.82,71.82,50.24,24.27,15.41,84.426042,36.197804,75.117173,46.691169,0,30,2,0,71.82,50.24,70.128770,68.440114,71.061146,-3.379886
3,2025-01-08 00:45:00,2025-01-08 01:00:00,BZN|PL,Without Sequence,71.82,71.82,50.24,24.27,15.41,84.921354,35.687726,75.201116,46.634338,0,45,2,0,71.82,50.24,70.128770,68.440114,71.079290,-3.379886
4,2025-01-08 01:00:00,2025-01-08 01:15:00,BZN|PL,Without Sequence,65.79,71.82,71.82,26.11,2.19,85.416667,35.163201,75.285060,46.577287,1,0,2,0,65.79,71.82,67.628028,68.483161,68.926332,2.693161


In [39]:
# Step 10.2: define peak and valley periods
q90 = results_2025["y_true"].quantile(0.9)
q10 = results_2025["y_true"].quantile(0.1)

peak_df = results_2025[results_2025["y_true"] >= q90]
valley_df = results_2025[results_2025["y_true"] <= q10]

print("Peak hours:", peak_df.shape[0])
print("Valley hours:", valley_df.shape[0])


Peak hours: 3303
Valley hours: 3302


In [40]:
# Step 10.3: error analysis
from sklearn.metrics import mean_absolute_error

mae_peak = mean_absolute_error(peak_df["y_true"], peak_df["y_pred_lgb_tuned"])
mae_valley = mean_absolute_error(valley_df["y_true"], valley_df["y_pred_lgb_tuned"])

mean_error_peak = (peak_df["y_pred_lgb_tuned"] - peak_df["y_true"]).mean()
mean_error_valley = (valley_df["y_pred_lgb_tuned"] - valley_df["y_true"]).mean()

print("Peak MAE:", mae_peak)
print("Valley MAE:", mae_valley)

print("Mean error (Peak):", mean_error_peak)
print("Mean error (Valley):", mean_error_valley)


Peak MAE: 15.06206981365496
Valley MAE: 18.91614631467903
Mean error (Peak): -4.967759129827719
Mean error (Valley): 18.276414427407172


In [41]:
# Step 10.4: hourly error profile
hourly_mae = (
    results_2025
    .groupby("hour")
    .apply(lambda x: mean_absolute_error(x["y_true"], x["y_pred_lgb_tuned"]))
)

print(hourly_mae)


hour
0      3.696420
1      2.459622
2      1.802573
3      1.324462
4      1.959044
5      3.207346
6      5.868559
7      6.768577
8      8.284034
9      9.468549
10     9.350961
11     8.670163
12     9.400719
13    10.391002
14    10.825252
15    10.794314
16     9.199759
17     9.309991
18     8.813507
19     9.923289
20    11.126904
21     6.958258
22     5.408796
23     3.578226
dtype: float64


C:\Users\achen\AppData\Local\Temp\ipykernel_22692\185736053.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: mean_absolute_error(x["y_true"], x["y_pred_lgb_tuned"]))
